# Day 24

## Part I

倒数第二天的问题，比起Day 22和Day 23来说略显简单，第一部分没什么好说的，就是一个坐标转换问题。这里使用复数作为六角网格中心点的坐标表示，但是有一个需要注意的地方，就是沿着斜线方向和沿着左右方向的步长理论上都是一样的，按照题目意思，应该就是正六边形的边长，但是如果需要使用实际数值的话，这里会涉及根号3，也就是sin(60)，如果使用浮点数，因为数值计算的局限性，将会有可能导致哈希的误差，也可以使用两个自定义的单位来代表sin(60)和cos(60)，然后自行实现坐标系，但是这使得问题复杂化了。因此我在第一部分中，假定六角网格不是正六边形，每次朝南北方向步进都是1，每次朝东西方向步进都是0.5，这样就能满足坐标转换的要求。

下面首先实现一个函数parse_step从字符串中依次读取每个（或每两个字符）作为下一步的指令，并返回下一步位移的复数量，南北移动单位为1，东西移动单位为0.5，并且同时返回本次解析读取了字符串中的几个字符。to_path_end函数将一行指令使用parse_step函数将每一步分解出来，然后使用复数加法算出最终格子的坐标并返回：

In [1]:
from typing import Tuple

def parse_step(steps: str) -> Tuple[complex, int]:
    if steps[0] == 'n':
        if steps[1] == 'w':
            return -.5-1j, 2
        if steps[1] == 'e':
            return .5-1j, 2
    if steps[0] == 's':
        if steps[1] == 'w':
            return -.5+1j, 2
        if steps[1] == 'e':
            return .5+1j, 2
    if steps[0] == 'w':
        return -1+0j, 1
    if steps[0] == 'e':
        return 1+0j, 1
    return None, -1

def to_path_end(line: str) -> complex:
    ref_tile = 0 + 0j
    while line:
        move_to, advance = parse_step(line)
        ref_tile += move_to
        line = line[advance:]
    return ref_tile

然后是读取输入文件，以及完成第一部分逻辑的代码，格子的颜色使用一个字典tiles记录，因为是二元的，因此使用bool类型表示，True为黑色，False为白色：

In [2]:
from typing import List, Dict

def read_input(input_file: str) -> List[str]:
    with open(input_file) as fn:
        return fn.readlines()

def count_all_black_tiles(tiles: Dict[complex, bool]) -> int:
    return sum(tiles.values())

def part1_solution(line_list: List[str]) -> Dict[complex, bool]:
    tiles = {}
    for line in line_list:
        end_tile = to_path_end(line.rstrip())
        tiles.setdefault(end_tile, False)
        tiles[end_tile] = not tiles[end_tile]
    return tiles

单元测试：

In [3]:
testcase = read_input('testcase1.txt')
test_tiles = part1_solution(testcase)
assert(count_all_black_tiles(test_tiles) == 10)

第一部分的结果：

In [4]:
lines = read_input('input.txt')
tiles = part1_solution(lines)
count_all_black_tiles(tiles)

339

## Part II

又是康威生命游戏变体。。。今年出现了三次。好吧，首先定义计算多少个黑色邻居的函数：

In [5]:
def count_black_neighbors(coord: complex, tiles: Dict[complex, bool]) -> int:
    diffs = (-.5-1j, -1+0j, -.5+1j, .5+1j, 1+0j, .5-1j) # 6个邻居的坐标差值
    counter = 0
    for d in diffs:
        neighbor = coord + d
        if tiles.get(neighbor, False):
            counter += 1
    return counter

新的一轮（一天）的计算函数，这里额外传递了所有黑色格子坐标的最左上角以及最右下角的两个坐标到函数，方便进行循环计算。然后在循环中重新设置最左上角和最右下角的坐标，用来代入下一次计算：

In [6]:
def next_day(tiles: Dict[complex, bool], 
             lt: complex, rb: complex) -> Tuple[Dict[complex, bool], complex, complex]:
    minx, miny = lt.real, lt.imag
    maxx, maxy = rb.real, rb.imag
    new_tiles = {}
    y = miny - 1. # y坐标最小值要减去单位 1
    while y < maxy + 2.: # y坐标最大值要加上单位 1
        x = minx - .5 # x 坐标最小值要减去单位0.5
        while x < maxx + 1.: # y坐标最大值要加上单位0.5
            coord = x + y * 1j
            black_neighbors = count_black_neighbors(coord, tiles)
            # 原本是黑色，并且黑色邻居数是1个或2个，保持黑色，此时不需要更新最大最小值
            if tiles.get(coord, False) and 0 < black_neighbors <= 2:
                new_tiles[coord] = True
            # 原本是白色，并且黑色邻居数是2个，变为黑色，此时需要更新最大最小值
            if not tiles.get(coord, False) and black_neighbors == 2:
                new_tiles[coord] = True
                if coord.real < minx:
                    lt = coord.real + lt.imag * 1j
                if coord.imag < miny:
                    lt = lt.real + coord.imag * 1j
                if coord.real > maxx:
                    rb = coord.real + rb.imag * 1j
                if coord.imag > maxy:
                    rb = rb.real + coord.imag * 1j
            x += .5
        y += 1.
                    
    return new_tiles, lt, rb

第二部分实现逻辑：

In [7]:
def part2_solution(lines: str, days: int=100) -> int:
    # 首先借用第一部分逻辑获得tiles的初始状态
    tiles = part1_solution(lines)
    # 求出最左上角和最右下角坐标
    x_list = [tile.real for tile in tiles]
    y_list = [tile.imag for tile in tiles]
    lt = min(x_list) + min(y_list) * 1j
    rb = max(x_list) + max(y_list) * 1j
    for _ in range(days):
        tiles, lt, rb = next_day(tiles, lt, rb)
    return count_all_black_tiles(tiles)

单元测试：

In [8]:
assert(part2_solution(testcase) == 2208)

第二部分结果：

In [9]:
part2_solution(lines)

3794